In [2]:
import os
os.chdir('F:/Projects/ML/chickenshit-classification')
%pwd

'F:\\Projects\\ML\\chickenshit-classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path=CONFIG_FILE_PATH, params_filepath: Path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        data_ingestion_config = self.config.data_ingestion
        create_directories([data_ingestion_config.root_dir])

        return DataIngestionConfig(
            root_dir=data_ingestion_config.root_dir,
            source_URL=data_ingestion_config.source_URL,
            local_data_file=data_ingestion_config.local_data_file,
            unzip_dir=data_ingestion_config.unzip_dir
        )

In [6]:
import os 
import urllib.request as request
import zipfile
from cnnClassifier.utils.common import get_size
from cnnClassifier import logger



In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_URL}")
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
                )
            logger.info(f"Downloaded file: {filename} with headers: \n{headers}")
        else:
            logger.info(f"Data file already exists at {self.config.local_data_file}. Size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted files to {self.config.unzip_dir}")

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e: 
    logger.exception(e)
    raise e 

[2024-07-06 14:28:02,021] [INFO]: common: Loaded yaml file from config\config.yaml
[2024-07-06 14:28:02,022] [INFO]: common: Loaded yaml file from params.yaml
[2024-07-06 14:28:02,023] [INFO]: common: Created directory: artifacts
[2024-07-06 14:28:02,024] [INFO]: common: Created directory: artifacts/data_ingestion
[2024-07-06 14:28:02,025] [INFO]: 2944837839: Data file already exists at artifacts/data_ingestion/data.zip. Size: ~ 11344.64 KB
[2024-07-06 14:28:02,256] [INFO]: 2944837839: Extracted files to artifacts/data_ingestion
